In [1]:
# !pip install -r requirements.sbv.txt

In [2]:
# !conda install -y -c conda-forge kaldi kalpy pynini

In [3]:
# !pip list

In [4]:
# # # 日本語辞書のダウンロード
# !mfa model download dictionary japanese_mfa

# # 日本語音響モデルのダウンロード
# !mfa model download acoustic japanese_mfa

In [5]:
import os
from typing import Literal
import ast

from dotenv import load_dotenv
from langchain_chroma import Chroma
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PDFMinerLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI


# .envファイル読み込み
load_dotenv("/users/s1f102201582/projects/mhcc-moshi/.env")

/nfs1/s1f102201582/anaconda3/envs/mfa_py311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [6]:
#config
from os.path import join, expanduser

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
BASE_URL = "https://api.openai.iniad.org/api/v1"
MODEL='gemini-2.5-flash'
TEMPERATURE = 1.0
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,4"

# 生成する音声のサンプリングレート
setting_sr = 16000

#対話音声データの個数を指定
gen_dial_num = 1

# すでに作成した対話データを削除するかどうか
IS_REMOVE_EXIST_FILE = True

# mfa関連のパス
model_dir = "/users/s1f102201582/Documents/MFA/pretrained_models/acoustic/japanese_mfa.zip"

#RAGで読み取るPDFのパス
rag_pdf_dir = "/users/s1f102201582/projects/mhcc-moshi/mental_docs/"

In [7]:
# model定義
model = ChatGoogleGenerativeAI(
                 model=MODEL,
                 temperature=TEMPERATURE)

# 埋め込みモデル定義
embeddings = OpenAIEmbeddings(
    openai_api_key=OPENAI_API_KEY,
    openai_api_base=BASE_URL,
    model="text-embedding-3-large"
)

# データベース定義
vector_store = Chroma(
    collection_name="collection",
    embedding_function=embeddings,
    # persist_directory = "/path/to/db_file" # if necessary
)

In [8]:
loader = DirectoryLoader(
    rag_pdf_dir,
    glob="*.pdf",
    show_progress=True,
    loader_cls=PDFMinerLoader,
)
docs = loader.load()
print(f"Loaded {len(docs)} documents")

  0%|                                                                                             | 0/3 [00:00<?, ?it/s]Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cannot set gray non-stroke color because /'P0' is an invalid float value
Cannot set gray non-stroke color because /'P1' is an invalid float value
Cann

Loaded 3 documents


In [9]:
#読み込んだ文章データをオーバーラップ200文字で1000文字づつ分割
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    add_start_index=True, # 分割前の文章のインデックスを追跡
)
splits = text_splitter.split_documents(docs)

# データベースにデータを追加
document_ids = vector_store.add_documents(documents=splits)

In [10]:
from langchain.agents.middleware import dynamic_prompt, ModelRequest

@dynamic_prompt
def prompt_with_context(request: ModelRequest) -> str:
    """Inject context into state messages."""
    last_query = request.state["messages"][-1].text
    retrieved_docs = vector_store.similarity_search(last_query, k=2)

    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

    system_message = (
        "You are a helpful assistant. Use the following context in your response:"
        f"\n\n{docs_content}"
    )

    return system_message

In [11]:
from typing import Literal

from pydantic import BaseModel, Field


class Dialogue(BaseModel):
    """対話データを構成する対話クラス"""
    speaker: Literal["A", "B"] = Field(..., description="話者。Aはカウンセラー、Bはクライエントを表す。")
    text: str = Field(..., description="話者が話した内容。")

class Dialogues(BaseModel):
    """カウンセリングを目的としたカウンセリング対話データ"""
    dialogues: list[Dialogue] = Field(..., description="対話データを構成する対話クラスのリスト。")

In [12]:
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy

agent = create_agent(
    model, 
    tools=[],
    middleware=[prompt_with_context],
    response_format=ToolStrategy(
        Dialogues,
        handle_errors="フォーマットに合うように、もう一度対話データを生成してください。"
    )
)

In [13]:
#promptを作成
import random


sessions = [
    "【段階：初期】信頼関係を築きつつ、悩みの背景を深掘りするシーン",
    "【段階：中期】クライエントの「すべき思考」に焦点を当て、認知の歪みを扱うシーン",
    "【段階：終結期】これまでのセッションを振り返り、終結に向けて準備するシーン",
]

def gen_prompt_txt():
    choiced = random.randint(0, 2)
    choiced_session = sessions[choiced]
    prompt_txt = f"""メンタルヘルスケアカウンセリングのセッションをシミュレーションしてください。
シミュレーションしたい「段階」と「テーマ」:
{choiced_session}

役割定義:
A (カウンセラー): メンタルヘルスケアの専門知識を持つ経験豊富なカウンセラー。傾聴と共感の姿勢を基本とし、クライエントの言葉を促すように、優しく、自然な話し言葉（「〜ですね」「〜でしたか」など）を使います。
B (クライエント): 仕事上の悩みだけでなく、日常生活全般に対して漠然とした不安や焦りを感じている人物。

対話の要件:
スタイル: 実際の会話の文字起こしのように、堅苦しくない自然な「話し言葉」を使用してください。
相槌 (あいづち): カウンセラー（A）は、クライエント（B）の話を促し、共感を示すため、「ええ」「はい」「そうなんですね」「なるほど」といった細かな相槌を頻繁に、適切なタイミングで挿入してください。
構成: 会話が途中で途切れるのではなく、初回のヒアリングとして「一区切り」がつき、自然に終了する流れにしてください（例：次回の約束、今回のまとめなど）。
分量: 会話の往復は合計12〜20ターン程度、全体の文字数が合計500〜800文字程度になるように構成してください。
"""
    return prompt_txt

In [14]:
# テキスト対話生成関数
def gen_txt_dialogue():
    prompt = gen_prompt_txt()
    resp = agent.invoke({"messages": [{"role": "user", "content": prompt}]})
    dialogues_list = resp["structured_response"].dialogues
    return dialogues_list

In [15]:
from style_bert_vits2.nlp import bert_models
from style_bert_vits2.constants import Languages
from pathlib import Path
from huggingface_hub import hf_hub_download
from style_bert_vits2.tts_model import TTSModel

bert_models.load_model(Languages.JP, "ku-nlp/deberta-v2-large-japanese-char-wwm")
bert_models.load_tokenizer(Languages.JP, "ku-nlp/deberta-v2-large-japanese-char-wwm")
assets_root = Path("model_assets")

# # 子春音あみ
# model_file = "koharune-ami/koharune-ami.safetensors"
# config_file = "koharune-ami/config.json"
# style_file = "koharune-ami/style_vectors.npy"
# hf_repo = "litagin/sbv2_koharune_ami"

# # あみたろ
# model_file = "amitaro/amitaro.safetensors"
# config_file = "amitaro/config.json"
# style_file = "amitaro/style_vectors.npy"
# hf_repo = "litagin/sbv2_amitaro"


# デフォルトの女性2
model_file = "jvnv-F2-jp/jvnv-F2_e166_s20000.safetensors"
config_file = "jvnv-F2-jp/config.json"
style_file = "jvnv-F2-jp/style_vectors.npy"
hf_repo = "litagin/style_bert_vits2_jvnv"

for file in [model_file, config_file, style_file]:
    print(file)
    hf_hub_download(hf_repo, file, local_dir="model_assets")

A_model = TTSModel(
    model_path=assets_root / model_file,
    config_path=assets_root / config_file,
    style_vec_path=assets_root / style_file,
    device="cuda",
)

# デフォルトの男性2
model_file = "jvnv-M2-jp/jvnv-M2-jp_e159_s17000.safetensors"
config_file = "jvnv-M2-jp/config.json"
style_file = "jvnv-M2-jp/style_vectors.npy"

for file in [model_file, config_file, style_file]:
    print(file)
    hf_hub_download(hf_repo, file, local_dir="model_assets")

B_model = TTSModel(
    model_path=assets_root / model_file,
    config_path=assets_root / config_file,
    style_vec_path=assets_root / style_file,
    device="cuda",
)

11-05 23:34:12 |  INFO  | bert_models.py:92 | Loaded the Languages.JP BERT model from ku-nlp/deberta-v2-large-japanese-char-wwm
11-05 23:34:13 |  INFO  | bert_models.py:154 | Loaded the Languages.JP BERT tokenizer from ku-nlp/deberta-v2-large-japanese-char-wwm
jvnv-F2-jp/jvnv-F2_e166_s20000.safetensors
jvnv-F2-jp/config.json
jvnv-F2-jp/style_vectors.npy
jvnv-M2-jp/jvnv-M2-jp_e159_s17000.safetensors
jvnv-M2-jp/config.json
jvnv-M2-jp/style_vectors.npy


In [16]:
def build_audio_synth_prompt(text_dialogue_list):
    resp = ""
    resp_header =  """あなたがこれから音声合成するテキストは以下の対話内容のワンフレーズです。
この対話の文脈に合うように音声合成してください。

<対話内容の全文>"""
    resp += resp_header
    for text_dial in text_dialogue_list:
        resp += f"\n{text_dial.speaker}: {text_dial.text}"
    return resp

In [17]:
from typing import Literal

def sbv_tts(text: str, speaker: Literal["A", "B"], assist_text=None):
    if speaker == "A":
        sr, audio = A_model.infer(
            text = text,
            style='Happy',
            style_weight=1,
            split_interval = 0.3,
            use_assist_text = True if assist_text is not None else None,
            assist_text = assist_text
        )
    else:
        sr, audio = B_model.infer(
            text = text,
            style='Sad',
            style_weight=1,
            split_interval = 0.3,
            use_assist_text = True if assist_text is not None else None,
            assist_text = assist_text
        )
    
    return sr, audio

In [18]:
import librosa
import numpy as np

def gen_audio_dialogue(text_dialogue_list, prompt):
    # 音声ファイルを順番に生成（ファイルは不要なのでwave配列で持つ）
    wav_data = []
    for dial in text_dialogue_list:
        speaker = dial.speaker
        sr, wav = sbv_tts(dial.text, speaker, prompt)

        # サンプリングレートを変換
        if sr != setting_sr:
            # 16ビット整数のデータを、-1.0から1.0の範囲に収まる浮動小数点数に正規化
            wav = wav.astype(np.float32) / 32768.0
            wav = librosa.resample(wav, orig_sr=sr, target_sr=setting_sr)

        # 0.3秒間の無音時間を追加
        duration_sec = 0.3
        num_silent_samples = int(setting_sr*duration_sec)
        silence = np.zeros(num_silent_samples, dtype=wav.dtype)
        wav_with_silence = np.concatenate((wav, silence))
        wav_data.append(wav_with_silence)
    
    # 最終的な音声長を決定
    max_len = sum([len(w) for w in wav_data])
    
    # ステレオ音声用（2チャンネル×最大長）の空配列をゼロ初期化で作成
    stereo = np.zeros((2, max_len), dtype=np.float32)
    
    pos = 0
    for i, wav in enumerate(wav_data):
        ch = i%2  # 0:左(A), 1:右(B)
        stereo[ch, pos:pos+len(wav)] += wav
        pos += len(wav)
    
    # 転置(-1,2)する
    stereo = stereo.T
    return stereo

In [19]:
def write_text(file_name, text_dialogue_list):
    result = ""
    for dial in text_dialogue_list:
        result += dial.text + "\n"
    with open(file_name, "w") as f:
        f.write(result)
    return

In [20]:
import soundfile as sf

text_dialogue_list = gen_txt_dialogue()
audio_synth_prompt = build_audio_synth_prompt(text_dialogue_list)
# 対話テキストを音声合成
stereo = gen_audio_dialogue(text_dialogue_list, audio_synth_prompt)

wav_name = "./mfa_test_in/test.wav"
# wavファイル出力
sf.write(wav_name, stereo, setting_sr)

text_file_name = "./mfa_test_in/test.txt"
write_text(text_file_name, text_dialogue_list)

11-05 23:34:28 |  INFO  | tts_model.py:259 | Start generating audio data from text:
こんにちは、Bさん。今日はどんな様子でしたか？何か気になっていることはありますか？
11-05 23:34:28 |  INFO  | infer.py:24 | Using JP-Extra model


/nfs1/s1f102201582/anaconda3/envs/mfa_py311/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


11-05 23:34:36 |  INFO  | safetensors.py:50 | Loaded 'model_assets/jvnv-F2-jp/jvnv-F2_e166_s20000.safetensors' (iteration 166)


/nfs1/s1f102201582/anaconda3/envs/mfa_py311/lib/python3.11/site-packages/pyopenjtalk/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [21]:
%%time

import subprocess
from os.path import expanduser, join

home_dir = expanduser("~")
model_dir = join(home_dir, "Documents/MFA/pretrained_models/acoustic/japanese_mfa.zip")

subprocess.run([
    "mfa",
    "align",
    "--quiet",
    "--fine_tune", 
    "--overwrite",
    "--clean",
    "--final_clean",
    "--output_format", "json",
    "./mfa_test_in/",
    "japanese_mfa",
    model_dir,
    "./mfa_test_out/",
    "--beam", "1000",
    "--retry_beam", "4000",
    "--punctuation", '"…"',
])

    

 INFO     Setting up corpus information...                                      
 INFO     Loading corpus from source files...                                   
 INFO     Found 1 speaker across 1 file, average number of utterances per       
          speaker: 1.0                                                          
 INFO     Initializing multiprocessing jobs...                                  
 WARNING  Number of jobs was specified as 3, but due to only having 1 speakers, 
          MFA will only use 1 jobs. Use the --single_speaker flag if you would  
          like to split utterances across jobs regardless of their speaker.     
 INFO     Normalizing text...                                                   
 INFO     Generating MFCCs...                                                   
 INFO     Calculating CMVN...                                                   
 INFO     Generating final features...                                          
 INFO     Creating corpus sp

CPU times: user 230 ms, sys: 106 ms, total: 337 ms
Wall time: 2min 57s


CompletedProcess(args=['mfa', 'align', '--quiet', '--fine_tune', '--overwrite', '--clean', '--final_clean', '--output_format', 'json', './mfa_test_in/', 'japanese_mfa', '/users/s1f102201582/Documents/MFA/pretrained_models/acoustic/japanese_mfa.zip', './mfa_test_out/', '--beam', '1000', '--retry_beam', '4000', '--punctuation', '"…"'], returncode=0)

In [ ]:
# !mfa validate --ignore_acoustics ./test/ japanese_mfa